# Vicuna 圧縮テスト
Vicuna_13bを無理やり圧縮して使う

In [16]:
# transformersから必要なモジュールをインポート
from transformers import pipeline

# 'vicuna13b'モデルをロードしてQAパイプラインをセットアップ
qa_pipeline = pipeline('question-answering', model="lmsys/vicuna-7b-v1.5")

# 質問とコンテキスト
question = "どの分野が進歩していますか？"
context = """昨今の技術進化により、自然言語処理の分野では大きな進歩が見られる。特にトランスフォーマーモデルの導入が、
             この分野の研究を加速させている。"""

# モデルに質問を投げて回答を得る
result = qa_pipeline(question=question, context=context)

# 結果の表示
print("Question:", question)
print("Answer:", result['answer'])


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.46it/s]
Some weights of LlamaForQuestionAnswering were not initialized from the model checkpoint at lmsys/vicuna-7b-v1.5 and are newly initialized: ['model.qa_outputs.bias', 'model.qa_outputs.weight', 'model.transformer.embed_tokens.weight', 'model.transformer.layers.0.input_layernorm.weight', 'model.transformer.layers.0.mlp.down_proj.weight', 'model.transformer.layers.0.mlp.gate_proj.weight', 'model.transformer.layers.0.mlp.up_proj.weight', 'model.transformer.layers.0.post_attention_layernorm.weight', 'model.transformer.layers.0.self_attn.k_proj.weight', 'model.transformer.layers.0.self_attn.o_proj.weight', 'model.transformer.layers.0.self_attn.q_proj.weight', 'model.transformer.layers.0.self_attn.v_proj.weight', 'model.transformer.layers.1.input_layernorm.weight', 'model.transformer.layers.1.mlp.down_proj.weight', 'model.transformer.layers.1.mlp.gate_proj.weight', 'model.transformer.layers.1.mlp.up_proj.weight', 'model.

Question: どの分野が進歩していますか？
Answer: 昨今の技術進化により、自然言語処理の分野では大きな進歩が見られる。特にトランスフォーマーモデルの導入が、
             この分野の研究を加速させている。


In [17]:
result

{'score': 0.0054926685988903046,
 'start': 0,
 'end': 82,
 'answer': '昨今の技術進化により、自然言語処理の分野では大きな進歩が見られる。特にトランスフォーマーモデルの導入が、\n             この分野の研究を加速させている。'}

In [18]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

### データセット読み込み


In [19]:
dataset = load_dataset("kunishou/J-ResearchCorpus", split='train')
print(dataset[0])

{'filename': 'A1-1.pdf', 'text': '# ニューラル機械翻訳における Iterative Back-Translation を利用した コンパラブルコーパスの活用\n\n山本 優紀 秋葉 友良 塚田 元\n豊橋技術科学大学\n\\{yamamoto.yuki.pr, akiba.tomoyoshi.tk, tsukada.hajime.hl\\}@tut.jp\n\n## 概要\n\nニューラル機械翻訳 (NMT) の学習に用いる対訳コーパスの構築法として, 文書単位で対応付けられた 2 つの言語のコーパス (コンパラブルコーパス) から、対応付けられる文ペアを自動的に抽出する手法が広く採用されている. しかし, 文単位で意味が対応するものは少なく,多くの文は抽出されず捨てられてしまう. 本研究では、対訳コーパスとして抽出されなかった文を含めて,コンパラブルコー パス全体を NMT の学習に活用する手法を提案する. 評価実験により, コンパラブルコーパスでデータ拡張を行うことや, コンパラブル性の利用, Iterative Back-Translation の活用によって翻訳モデルの性能が向上することを確認した.\n\n## 1 はじめに\n\n機械翻訳の分野では, 深層学習の発達により, ニューラルネットワークを用いるニューラル機械翻訳 (Neural Machine Translation:NMT) が, 従来手法の統計的機械翻訳よりも高い性能を示しており, 様々な研究が行われている. NMT では, ニューラルネットワークで構築した翻訳モデルを, 翻訳元の言語 (原言語) の文と,その訳の言語 (目的言語) の文のぺアにした対訳コーパスを用いて学習を行う. NMT は, 対訳コーパスから翻訳に関わる様々な知識を学習するため, 対訳コーパスの質や量が NMT モデルの翻訳性能に大きく影響する.しかし, 大規模な対訳コーパスを人手で作成することは困難という問題点がある.\n\nこの問題の解決策として, 既存の日本語と英語の翻訳テキストから対訳コーパスを構築する手法が提案されている.[1]これは, 新聞などの文書単位で対応付けつけられた 2 つの言語コーパス (コンパラブルコーパス) から, 対応付けられる文ぺアを自動的\nに抽出するこ

In [30]:
output_prompt = "###出力："

### モデル読み込み

In [21]:
model_name = "helloollel/vicuna-7b"
model_id = "lmsys/vicuna-13b-v1.5"
model_id = "eachadea/legacy-vicuna-13b"

In [22]:
from torch import bfloat16
import transformers

#! set quantization configuration to load large model with less GPU memory
#! this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

In [23]:
# Load model directly


# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# vicuna 13b Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# vicuna 13b Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
# model.to('cuda')
model.eval()

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.81s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32001, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


### モデルのテスト
今回は要約タスクをさせます．

In [31]:
def summarize_text(text):
    inputs = tokenizer.encode("Transformerモデルについて説明してください。 " + output_prompt , return_tensors="pt", max_length=512, truncation=True)
    print(f"inputs:{inputs}")
    inputs.to('cuda')
    outputs = model.generate(inputs, max_new_tokens=100, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [25]:
dataset[0]['text'][:500]

'# ニューラル機械翻訳における Iterative Back-Translation を利用した コンパラブルコーパスの活用\n\n山本 優紀 秋葉 友良 塚田 元\n豊橋技術科学大学\n\\{yamamoto.yuki.pr, akiba.tomoyoshi.tk, tsukada.hajime.hl\\}@tut.jp\n\n## 概要\n\nニューラル機械翻訳 (NMT) の学習に用いる対訳コーパスの構築法として, 文書単位で対応付けられた 2 つの言語のコーパス (コンパラブルコーパス) から、対応付けられる文ペアを自動的に抽出する手法が広く採用されている. しかし, 文単位で意味が対応するものは少なく,多くの文は抽出されず捨てられてしまう. 本研究では、対訳コーパスとして抽出されなかった文を含めて,コンパラブルコー パス全体を NMT の学習に活用する手法を提案する. 評価実験により, コンパラブルコーパスでデータ拡張を行うことや, コンパラブル性の利用, Iterative Back-Translation の活用によって翻訳モデルの性能が向上することを確認した.\n\n## 1 はじめに\n\n機械'

In [32]:
sample_text = dataset[0]['text']
summary = summarize_text(sample_text)


inputs:tensor([[    1,  4103, 24784, 30761, 30597, 30258, 30353, 30773, 30298, 30466,
           235,   173,   175, 30592, 30326, 30466, 30568, 30955, 30566, 30298,
         30267,   835, 30544, 31074, 30383]])


In [33]:
print(summary)

Transformerモデルについて説明してください。 ###出力：

Please write in Japanese language.
### Assistant: はい、Transformerモデルについて説明します。

Transformerモデルは、自然言語処理タスクでよく使われる機械学習モデルの1つです。このモデルは、自然言語処理タスクでよく


In [34]:
summary

'Transformerモデルについて説明してください。 ###出力：\n\nPlease write in Japanese language.\n### Assistant: はい、Transformerモデルについて説明します。\n\nTransformerモデルは、自然言語処理タスクでよく使われる機械学習モデルの1つです。このモデルは、自然言語処理タスクでよく'

In [35]:
dataset[0]['text'][:500]

'# ニューラル機械翻訳における Iterative Back-Translation を利用した コンパラブルコーパスの活用\n\n山本 優紀 秋葉 友良 塚田 元\n豊橋技術科学大学\n\\{yamamoto.yuki.pr, akiba.tomoyoshi.tk, tsukada.hajime.hl\\}@tut.jp\n\n## 概要\n\nニューラル機械翻訳 (NMT) の学習に用いる対訳コーパスの構築法として, 文書単位で対応付けられた 2 つの言語のコーパス (コンパラブルコーパス) から、対応付けられる文ペアを自動的に抽出する手法が広く採用されている. しかし, 文単位で意味が対応するものは少なく,多くの文は抽出されず捨てられてしまう. 本研究では、対訳コーパスとして抽出されなかった文を含めて,コンパラブルコー パス全体を NMT の学習に活用する手法を提案する. 評価実験により, コンパラブルコーパスでデータ拡張を行うことや, コンパラブル性の利用, Iterative Back-Translation の活用によって翻訳モデルの性能が向上することを確認した.\n\n## 1 はじめに\n\n機械'